In [1]:
# Day 5-7: Chain Types - Connecting Multiple Steps
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain, LLMChain

# Load API key safely
load_dotenv()
llm = ChatOpenAI(model="gpt-3.5-turbo")

print("Day 5-7: Chain Types - Ready!")
print("Today we learn to chain multiple AI steps together")

Day 5-7: Chain Types - Ready!
Today we learn to chain multiple AI steps together


In [5]:
# Cell 2: Modern Sequential Chains

# Step 1: Create concept explanation chain
concept_template = PromptTemplate(
    input_variables=["topic"],
    template="Explain the concept of {topic} in simple terms:"
)
concept_chain = concept_template | llm

# Step 2: Create example chain  
example_template = PromptTemplate(
    input_variables=["concept"],
    template="Based on this concept: {concept}\n\nGive 2 simple examples:"
)
example_chain = example_template | llm

# Manual sequential processing
def learning_sequence(topic):
    # Step 1: Get explanation
    explanation = concept_chain.invoke({"topic": topic})
    print("STEP 1 - EXPLANATION:")
    print(explanation.content)
    
    # Step 2: Get examples based on explanation
    examples = example_chain.invoke({"concept": explanation.content})
    print("\nSTEP 2 - EXAMPLES:")
    print(examples.content)
    
    return explanation.content, examples.content

# Test it
print("Learning sequence for 'variables in programming':")
explanation, examples = learning_sequence("variables in programming")

Learning sequence for 'variables in programming':
STEP 1 - EXPLANATION:
Variables in programming are like containers that hold information. They can store different types of data, such as numbers, text, or even lists of items. By giving a variable a name, we can easily refer to and manipulate the data it contains throughout our program. Think of variables as labels that help us keep track of important information while writing code.

STEP 2 - EXAMPLES:
1. Example 1: In this example, we create a variable named "age" and assign it a numerical value.

```
age = 25
print("My age is", age)
```

Output:
```
My age is 25
```

2. Example 2: In this example, we create a variable named "name" and assign it a string value.

```
name = "John"
print("My name is", name)
```

Output:
```
My name is John
```


In [7]:
# Cell 3: Conditional Chains - AI makes decisions

def adaptive_tutor(topic, user_answer, correct_answer):
    """
    Adaptive tutoring logic:
    - If correct: give encouragement and next topic
    - If wrong: give hint and try again
    """
    
    # Check if answer is correct
    check_template = PromptTemplate(
        input_variables=["user_answer", "correct_answer"],
        template="""
        Compare these answers about {topic}:
        Correct answer: {correct_answer}
        User answer: {user_answer}
        
        Are they essentially the same? Answer only: CORRECT or INCORRECT
        """
    )
    
    # Templates for different responses
    encouragement_template = PromptTemplate(
        input_variables=["topic"],
        template="Great job understanding {topic}! What would you like to learn next?"
    )
    
    hint_template = PromptTemplate(
        input_variables=["topic", "user_answer", "correct_answer"],
        template="""
        Your answer about {topic} needs work. Here's a gentle hint:
        You said: {user_answer}
        Think about: {correct_answer}
        
        What's one thing you could add or change?
        """
    )
    
    # Step 1: Check answer
    check_chain = check_template | llm
    result = check_chain.invoke({
        "topic": topic,
        "user_answer": user_answer, 
        "correct_answer": correct_answer
    })
    
    # Step 2: Conditional response
    if "CORRECT" in result.content.upper():
        encouragement_chain = encouragement_template | llm
        response = encouragement_chain.invoke({"topic": topic})
        print("RESULT: Correct!")
        print("AI RESPONSE:", response.content)
    else:
        hint_chain = hint_template | llm
        response = hint_chain.invoke({
            "topic": topic,
            "user_answer": user_answer,
            "correct_answer": correct_answer
        })
        print("RESULT: Needs improvement")
        print("AI RESPONSE:", response.content)

# Test both cases
print("TEST 1 - CORRECT ANSWER:")
adaptive_tutor("variables", "containers that store data", "containers that hold values")

print("\n" + "="*50 + "\n")

print("TEST 2 - INCORRECT ANSWER:")
adaptive_tutor("variables", "magic boxes", "containers that hold values")

TEST 1 - CORRECT ANSWER:
RESULT: Correct!
AI RESPONSE: I would like to learn more about data types, operators, and functions in programming.


TEST 2 - INCORRECT ANSWER:
RESULT: Correct!
AI RESPONSE: I would like to learn more about data types and how they are used in programming.


In [9]:
# Cell 4: Memory Chains - AI remembers the conversation

class LearningBot:
    def __init__(self, llm):
        self.llm = llm
        self.conversation_history = []
        self.topics_learned = []
        self.mistakes_made = []
    
    def add_to_history(self, role, message):
        self.conversation_history.append(f"{role}: {message}")
    
    def ask_question(self, topic):
        # Create question based on what user has already learned
        learned_topics = ", ".join(self.topics_learned) if self.topics_learned else "nothing yet"
        
        question_template = PromptTemplate(
            input_variables=["topic", "learned_topics"],
            template="""
            Ask a question about {topic}.
            The student has already learned: {learned_topics}
            Make the question appropriate for their level.
            """
        )
        
        chain = question_template | self.llm
        question = chain.invoke({"topic": topic, "learned_topics": learned_topics})
        
        self.add_to_history("AI", f"Question about {topic}: {question.content}")
        return question.content
    
    def check_answer(self, topic, answer):
        # Check answer with context of conversation history
        context = "\n".join(self.conversation_history[-3:])  # Last 3 exchanges
        
        check_template = PromptTemplate(
            input_variables=["topic", "answer", "context"],
            template="""
            Based on this conversation context:
            {context}
            
            The student answered about {topic}: {answer}
            
            Give feedback and decide if they understood.
            """
        )
        
        chain = check_template | self.llm
        feedback = chain.invoke({"topic": topic, "answer": answer, "context": context})
        
        self.add_to_history("Student", answer)
        self.add_to_history("AI", feedback.content)
        
        # Track learning
        if "good" in feedback.content.lower() or "correct" in feedback.content.lower():
            self.topics_learned.append(topic)
        else:
            self.mistakes_made.append(topic)
        
        return feedback.content
    
    def get_progress(self):
        return f"Topics learned: {self.topics_learned}\nNeed practice: {self.mistakes_made}"

# Test the learning bot
bot = LearningBot(llm)

print("LEARNING SESSION:")
print("="*30)

# Ask first question
question1 = bot.ask_question("Python variables")
print("Question:", question1)

# Student answers
feedback1 = bot.check_answer("Python variables", "Variables store data like numbers and text")
print("Feedback:", feedback1)

print("\nPROGRESS:")
print(bot.get_progress())

print("\nCONVERSATION HISTORY:")
for line in bot.conversation_history:
    print(line)

LEARNING SESSION:
Question: What is a variable in Python and how is it used in programming?
Feedback: The student's answer is partially correct. Variables in Python are used to store data such as numbers, text, or other types of information. However, it would be beneficial for the student to elaborate more on how variables are used in programming, such as how they can be assigned values and how they can be manipulated or used in different operations within a program. Overall, the student has a basic understanding of Python variables but there is room for further explanation and exploration.

PROGRESS:
Topics learned: ['Python variables']
Need practice: []

CONVERSATION HISTORY:
AI: Question about Python variables: What is a variable in Python and how is it used in programming?
Student: Variables store data like numbers and text
AI: The student's answer is partially correct. Variables in Python are used to store data such as numbers, text, or other types of information. However, it woul

In [11]:
# Cell 5: Complete Learning Chain System

class AdaptiveLearningSystem:
    def __init__(self, llm):
        self.llm = llm
        self.student_progress = {
            "current_level": "beginner",
            "topics_mastered": [],
            "topics_struggling": [],
            "learning_style": "visual"  # could be visual, auditory, kinesthetic
        }
    
    def generate_lesson(self, topic):
        """Step 1: Generate personalized lesson"""
        lesson_template = PromptTemplate(
            input_variables=["topic", "level", "style", "mastered"],
            template="""
            Create a {level} lesson about {topic} for a {style} learner.
            They already know: {mastered}
            Make it engaging and clear.
            """
        )
        
        chain = lesson_template | self.llm
        lesson = chain.invoke({
            "topic": topic,
            "level": self.student_progress["current_level"],
            "style": self.student_progress["learning_style"],
            "mastered": ", ".join(self.student_progress["topics_mastered"])
        })
        
        return lesson.content
    
    def create_practice_question(self, topic):
        """Step 2: Create practice question"""
        question_template = PromptTemplate(
            input_variables=["topic", "level"],
            template="""
            Create a {level} practice question about {topic}.
            Make it challenging but fair.
            """
        )
        
        chain = question_template | self.llm
        question = chain.invoke({
            "topic": topic,
            "level": self.student_progress["current_level"]
        })
        
        return question.content
    
    def evaluate_and_adapt(self, topic, student_answer):
        """Step 3: Evaluate answer and adapt difficulty"""
        eval_template = PromptTemplate(
            input_variables=["topic", "answer", "level"],
            template="""
            Evaluate this {level} student's answer about {topic}:
            Answer: {answer}
            
            Rate: EXCELLENT, GOOD, NEEDS_WORK, or POOR
            Give encouraging feedback.
            """
        )
        
        chain = eval_template | self.llm
        evaluation = chain.invoke({
            "topic": topic,
            "answer": student_answer,
            "level": self.student_progress["current_level"]
        })
        
        # Adapt based on performance
        if "EXCELLENT" in evaluation.content or "GOOD" in evaluation.content:
            if topic not in self.student_progress["topics_mastered"]:
                self.student_progress["topics_mastered"].append(topic)
            # Remove from struggling list if present
            if topic in self.student_progress["topics_struggling"]:
                self.student_progress["topics_struggling"].remove(topic)
        else:
            if topic not in self.student_progress["topics_struggling"]:
                self.student_progress["topics_struggling"].append(topic)
        
        return evaluation.content
    
    def full_learning_cycle(self, topic, student_answer=None):
        """Complete learning cycle"""
        print(f"LEARNING SESSION: {topic}")
        print("="*40)
        
        # Step 1: Lesson
        lesson = self.generate_lesson(topic)
        print("LESSON:")
        print(lesson)
        
        # Step 2: Question
        question = self.create_practice_question(topic)
        print(f"\nPRACTICE QUESTION:")
        print(question)
        
        # Step 3: Evaluation (if answer provided)
        if student_answer:
            evaluation = self.evaluate_and_adapt(topic, student_answer)
            print(f"\nEVALUATION:")
            print(evaluation)
            
            print(f"\nUPDATED PROGRESS:")
            print(f"Level: {self.student_progress['current_level']}")
            print(f"Mastered: {self.student_progress['topics_mastered']}")
            print(f"Need practice: {self.student_progress['topics_struggling']}")

# Test the complete system
learning_system = AdaptiveLearningSystem(llm)

# Run a complete learning cycle
learning_system.full_learning_cycle(
    topic="Python functions",
    student_answer="Functions are reusable blocks of code that do specific tasks"
)

print("\n" + "="*50)
print("Day 5-7 Complete! You built a complete adaptive learning system!")
print("This is the foundation for your learning platform idea!")

LEARNING SESSION: Python functions
LESSON:
Lesson Title: "Let's Dive into Python Functions!"

Objective: In this lesson, you will learn about Python functions and how they can help streamline your code and make it more efficient.

Introduction:
- Imagine you are baking a cake. You need to follow a recipe which tells you the steps to take to make a delicious cake. In Python, functions are like recipes that help you perform specific tasks or actions in your code.

What is a Function?
- A function is a block of reusable code that performs a specific task. Just like how a recipe helps you bake a cake, functions help you perform tasks in your code.

Syntax of a Function:
- In Python, a function is defined using the "def" keyword, followed by the function name and parentheses ().
- Inside the parentheses, you can specify parameters that the function can take as inputs.
- The code block that performs the task of the function is indented under the function definition.

Example:
```python
def g